In [1]:
## create ground truth label from batch results
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join


In [4]:
## Load the CSV files and save in 1 dataframe
csv_files = [f for f in listdir("batch_results") if isfile(join("batch_results", f))]

# all_results = pd.DataFrame(columns=["anchor_id", "anchor_label", image_id", "melanoma"])
all_results = pd.DataFrame()

for i in csv_files:
    temp_df = pd.read_csv("batch_results/"+i)
    all_results = all_results.append(temp_df, ignore_index=True)
    

# print(all_results.columns)

## Drop all rows we don't need
df_prep0 = all_results.drop(columns=['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.id',
       'Input.item1_url', 'Input.item2_label', 'Input.item2_url', 'Input.item3_label', 'Input.item3_url','Approve', 'Reject'])

# Rename columns
df_prep0 = df_prep0.rename(columns={"Input.item1_title":"anchor_id", "Input.item1_label" : "anchor_label", "Input.item2_title":"item2", "Input.item3_title":"item3", "Answer.equal.label":"answer"  })
print(df_prep0)

## Create the ground truth file
# Step 1: give image_id to picture that has been answered

df_prep1 = df_prep0
## Find image ID based on the answer is A or N
df_prep1[["image_id"]] = df_prep1[["item2"]].where(df_prep1[["answer"]].values=="A")
## Because it is binary, we now all NaN is B (Image 3)
df_prep1.image_id.fillna(df_prep1.item3,inplace=True)
#print(df_prep1)

# Step 2: define the label equal to the anchor
df_prep2 = pd.DataFrame()
df_prep2[["image_id", "melanoma"]] = df_prep1[["image_id", "anchor_label"]]
df_prep2 = df_prep2.sort_values("image_id")
print(df_prep2)

# Step 3: Take average of all answers 
df_prep3 = df_prep2
df_prep3[["melanoma"]] = df_prep3["melanoma"].astype(float)
df_prep3 = df_prep3.groupby("image_id").mean()
print(df_prep3)

# Step 4: Round to nearest label {0,1}
df = df_prep3
df = df.round({'melanoma': 0})
print(df)


       anchor_id  anchor_label         item2         item3 answer
0   ISIC_0013490             1  ISIC_5366635  ISIC_4329544      B
1   ISIC_0010025             0  ISIC_3542273  ISIC_8609756      A
2   ISIC_0013567             1  ISIC_9579703  ISIC_4508878      A
3   ISIC_0011230             0  ISIC_7035246  ISIC_6613150      A
4   ISIC_0009919             0  ISIC_3738751  ISIC_8806647      B
5   ISIC_0010015             0  ISIC_1109641  ISIC_4946006      A
6   ISIC_0013685             0  ISIC_9472515  ISIC_7730937      B
7   ISIC_0000409             0  ISIC_5332605  ISIC_5272223      B
8   ISIC_0000300             1  ISIC_5581297  ISIC_9152137      B
9   ISIC_0013815             1  ISIC_0632512  ISIC_1404148      B
10  ISIC_0013490             1  ISIC_5366635  ISIC_4329544      B
11  ISIC_0010025             0  ISIC_3542273  ISIC_8609756      A
12  ISIC_0013567             1  ISIC_9579703  ISIC_4508878      A
13  ISIC_0011230             0  ISIC_7035246  ISIC_6613150      A
14  ISIC_0

In [71]:
## Save ground truth file to csv
df.to_csv("CROWDSIM-2021_GroundTruth.csv")